In [1]:
#imports
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace, date_format
from pyspark.sql.types import IntegerType, DoubleType, FloatType

In [2]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatel0311"

In [3]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- id_folio: string (nullable = true)
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tipo_de_entrada: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- fecha_concluido: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- 0311_colonia_registro: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [5]:
print(f"Numero de renglones {df.count()}")
print(f"Numero de columnas {len(df.columns)}")
print(f"Columnas {df.columns}")

Numero de renglones 2323089
Numero de columnas 15
Columnas ['id_folio', 'fecha_solicitud', 'hora_solicitud', 'tipo_de_entrada', 'tema_solicitud', 'sexo', 'edad', 'estatus', 'fecha_concluido', 'alcaldia', 'colonia_datos', '0311_colonia_registro', 'codigo_postal_solicitud', 'latitud', 'longitud']


In [6]:
df.show(truncate=False)

+--------+---------------+--------------+---------------------+---------------------------------------+---------------+----+--------+---------------+--------+-------------+---------------------+-----------------------+-------+--------+
|id_folio|fecha_solicitud|hora_solicitud|tipo_de_entrada      |tema_solicitud                         |sexo           |edad|estatus |fecha_concluido|alcaldia|colonia_datos|0311_colonia_registro|codigo_postal_solicitud|latitud|longitud|
+--------+---------------+--------------+---------------------+---------------------------------------+---------------+----+--------+---------------+--------+-------------+---------------------+-----------------------+-------+--------+
|- 244841|2020-01-11     |08:17:58      |VICTORIA             |FUGA DE AGUA                           |NO ESPECIFICADO|NA  |ATENDIDO|2020-07-03     |NA      |NA           |NA                   |NA                     |0      |0       |
|- 265596|2020-01-28     |12:10:42      |VICTORIA       

In [7]:
df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo','edad','estatus','alcaldia','colonia_datos','codigo_postal_solicitud','latitud','longitud']).show()

+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-----------------------+-------+--------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|           sexo|edad| estatus|alcaldia|colonia_datos|codigo_postal_solicitud|latitud|longitud|
+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-----------------------+-------+--------+
|     2020-01-11|      08:17:58|        FUGA DE AGUA|NO ESPECIFICADO|  NA|ATENDIDO|      NA|           NA|                     NA|      0|       0|
|     2020-01-28|      12:10:42|VENTA DE ALCOHOL ...|NO ESPECIFICADO|  NA|ATENDIDO|      NA|           NA|                     NA|      0|       0|
|     2020-01-10|      12:39:20|TRAMITES / INFO V...|NO ESPECIFICADO|  NA|ATENDIDO|      NA|           NA|                     NA|      0|       0|
|     2020-01-24|      10:17:36|MANTENIMIENTO DRE...|NO ESPECIFICADO|  NA|ATENDIDO|      NA|           NA|      

In [8]:
datos = df.select(['fecha_solicitud','hora_solicitud','tema_solicitud','sexo','edad','estatus','alcaldia','colonia_datos','latitud','longitud'])

In [9]:
datos.printSchema()

root
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [10]:
datos.count()

2323089

In [11]:
datos.filter('sexo = "FEMENINO"').count()

829463

In [12]:
datos \
    .select(['fecha_solicitud','hora_solicitud','tema_solicitud','edad','estatus','alcaldia','colonia_datos']) \
    .filter('sexo = "FEMENINO"') \
    .orderBy('fecha_solicitud') \
    .show()

+---------------+--------------+--------------------+----+--------+-----------------+--------------------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|edad| estatus|         alcaldia|       colonia_datos|
+---------------+--------------+--------------------+----+--------+-----------------+--------------------+
|     2019-06-02|      16:50:47|           ALUMBRADO|  29|ATENDIDO|       CUAUHTEMOC|      ROMA NORTE III|
|     2019-06-02|      23:30:08|       FALTA DE AGUA|  50|ATENDIDO|               NA|                  NA|
|     2019-06-02|      17:28:06|      MEDIO AMBIENTE|  19|ATENDIDO|               NA|                  NA|
|     2019-06-02|      14:40:34|        FUGA DE AGUA|  38|ATENDIDO|          TLALPAN|FLORESTA-PRADO-VE...|
|     2019-06-02|      17:33:25|MANTENIMIENTO VÍA...|  71|ATENDIDO|       IZTAPALAPA|          MODELO (U)|
|     2019-06-02|      15:17:44|              BACHEO|  66|ATENDIDO|         COYOACAN|  AJUSCO HUAYAMILPAS|
|     2019-06-02|      17:40:36|     

In [13]:
#Transformar columnas (cast)
datos = datos.withColumn("fecha_solicitud", to_date(col("fecha_solicitud"), "yyyy-MM-dd")) \
        .withColumn("hora_solicitud", date_format(col("fecha_solicitud"), "HH:MM:SS")) \
        .withColumn("edad", col("edad").cast(FloatType())) \
        .withColumn("latitud", col("latitud").cast(DoubleType())) \
        .withColumn("longitud", col("longitud").cast(DoubleType())) 

datos.printSchema()
datos.show()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)

+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-------+--------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|           sexo|edad| estatus|alcaldia|colonia_datos|latitud|longitud|
+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-------+--------+
|     2020-01-11|      00:01:00|        FUGA DE AGUA|NO ESPECIFICADO|NULL|ATENDIDO|      NA|           NA|    0.0|     0.0|
|     2020-01-28|      00:01:00|VENTA DE ALCOHOL ...|NO ESPECIFICADO|NULL|ATENDIDO|    

In [14]:
#Contar nulos por columna
datos.select([sum(col(c).isNull().cast("int")).alias(c) for c in datos.columns]).show()

+---------------+--------------+--------------+----+-------+-------+--------+-------------+-------+--------+
|fecha_solicitud|hora_solicitud|tema_solicitud|sexo|   edad|estatus|alcaldia|colonia_datos|latitud|longitud|
+---------------+--------------+--------------+----+-------+-------+--------+-------------+-------+--------+
|              0|             0|             0|   0|1402897|      0|       0|            0| 682509|  431500|
+---------------+--------------+--------------+----+-------+-------+--------+-------------+-------+--------+



In [15]:
from pyspark.sql.functions import hour, minute, second, year, month, dayofmonth, weekofyear
colfs = "fecha_solicitud"
colhs = "hora_solicitud"

datos_tiempo = datos.select(
    col("edad"), col("estatus"), col("tema_solicitud"),
    year(col(colfs)).alias("anio"), 
    month(col(colfs)).alias("mes"),
    dayofmonth(col(colfs)).alias("dia"),
    weekofyear(col(colfs)).alias("semana"),
    date_format(col(colfs), "MMMM").alias("nombre_mes"),
    date_format(col(colfs), "EEEE").alias("nombre_dia"),
    hour(col(colhs)).alias("hora"),
    minute(col(colhs)).alias("minuto"),    
    second(col(colhs)).alias("segundo")
)

datos_tiempo.printSchema()

root
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)
 |-- semana: integer (nullable = true)
 |-- nombre_mes: string (nullable = true)
 |-- nombre_dia: string (nullable = true)
 |-- hora: integer (nullable = true)
 |-- minuto: integer (nullable = true)
 |-- segundo: integer (nullable = true)



In [16]:
datos_tiempo.show(truncate=False)

+----+--------+---------------------------------------+----+---+---+------+----------+----------+----+------+-------+
|edad|estatus |tema_solicitud                         |anio|mes|dia|semana|nombre_mes|nombre_dia|hora|minuto|segundo|
+----+--------+---------------------------------------+----+---+---+------+----------+----------+----+------+-------+
|NULL|ATENDIDO|FUGA DE AGUA                           |2020|1  |11 |2     |January   |Saturday  |0   |1     |0      |
|NULL|ATENDIDO|VENTA DE ALCOHOL / DROGA               |2020|1  |28 |5     |January   |Tuesday   |0   |1     |0      |
|NULL|ATENDIDO|TRAMITES / INFO VEHICULAR              |2020|1  |10 |2     |January   |Friday    |0   |1     |0      |
|NULL|ATENDIDO|MANTENIMIENTO DRENAJE                  |2020|1  |24 |4     |January   |Friday    |0   |1     |0      |
|NULL|ATENDIDO|SOLICITUD DE VIGILANCIA                |2020|1  |4  |1     |January   |Saturday  |0   |1     |0      |
|NULL|ATENDIDO|ALERTA SISMICA                         |2

In [17]:
datos_tiempo.count()

2323089

In [18]:
#estadisticos descriptivos del DF
datos_tiempo.describe().show()

+-------+------------------+--------------------+--------------------+------------------+------------------+-----------------+-----------------+----------+----------+-------+------------------+-------+
|summary|              edad|             estatus|      tema_solicitud|              anio|               mes|              dia|           semana|nombre_mes|nombre_dia|   hora|            minuto|segundo|
+-------+------------------+--------------------+--------------------+------------------+------------------+-----------------+-----------------+----------+----------+-------+------------------+-------+
|  count|            920192|             2323089|             2323089|           2323089|           2323089|          2323089|          2323089|   2323089|   2323089|2323089|           2323089|2323089|
|   mean| 39.27146399881764|                18.0|                NULL|2021.5103188039718|6.4875654785503265|15.85532538787795|26.43079107171529|      NULL|      NULL|    0.0|6.4875654785503265

In [19]:
#Estadisticos descriptivos de algunas columnas del DataFrame
columnas = ['edad', 'anio', 'mes', 'dia', 'hora', 'minuto', 'segundo', 'semana']
datos_tiempo.select(columnas).describe().show()

+-------+------------------+------------------+------------------+-----------------+-------+------------------+-------+-----------------+
|summary|              edad|              anio|               mes|              dia|   hora|            minuto|segundo|           semana|
+-------+------------------+------------------+------------------+-----------------+-------+------------------+-------+-----------------+
|  count|            920192|           2323089|           2323089|          2323089|2323089|           2323089|2323089|          2323089|
|   mean| 39.27146399881764|2021.5103188039718|6.4875654785503265|15.85532538787795|    0.0|6.4875654785503265|    0.0|26.43079107171529|
| stddev|27.238747481977125|  1.37871177376573|3.4064677225068465| 8.65088685805828|    0.0|3.4064677225068465|    0.0|14.82067661213932|
|    min|               0.0|              2019|                 1|                1|      0|                 1|      0|                1|
|    max|           16090.0|      

In [20]:
datos_tiempo.select(columnas).describe().toPandas().T

,0,1,2,3,4
summary,count,mean,stddev,min,max
edad,920192,39.27146399881764,27.238747481977125,0.0,16090.0
anio,2323089,2021.5103188039718,1.37871177376573,2019,2024
mes,2323089,6.4875654785503265,3.4064677225068465,1,12
dia,2323089,15.85532538787795,8.65088685805828,1,31
hora,2323089,0.0,0.0,0,0
minuto,2323089,6.4875654785503265,3.4064677225068465,1,12
segundo,2323089,0.0,0.0,0,0
semana,2323089,26.43079107171529,14.82067661213932,1,53


In [21]:
#Ver valores únicos
datos_tiempo.select("tema_solicitud").distinct().count()
datos_tiempo.select("tema_solicitud").distinct().show(truncate=False)

+--------------------------------+
|tema_solicitud                  |
+--------------------------------+
|VERIFICACIÓN ADMINISTRATIVA     |
|BACHEO                          |
|REPARACION DE EMPEDRADO         |
|BECAS                           |
|FALTA DE AGUA                   |
|MERCADOS PÚBLICOS, TIANGUIS     |
|REINSCRIPCION AL CENDI          |
|LLAVE CDMX                      |
|ASESORÍA DE TERCEROS ACREDITADOS|
|QUEJAS Y DENUNCIAS              |
|WIFI GRATIS                     |
|REGISTRO CIVIL                  |
|CONSTRUCCIONES Y OBRAS          |
|ALARMAS VECINALES               |
|SOLICITUD/SEGURO DE DESEMPLEO   |
|AVISO PREVIO DE MANIFESTACIONES |
|SOLICITUD DE VIGILANCIA         |
|FOMENTO ECONÓMICO Y EMPLEO      |
|PODA / RETIRO ARBOL             |
|INFORMACIÓN SOBRE TRÁMITES      |
+--------------------------------+
only showing top 20 rows



In [22]:
datos_tiempo.select("tema_solicitud").distinct().orderBy("tema_solicitud").show(100, truncate = False)

+----------------------------------------+
|tema_solicitud                          |
+----------------------------------------+
|AGUA POTABLE Y SERVICIOS HIDRÁULICOS    |
|ALARMAS VECINALES                       |
|ALERTA SISMICA                          |
|ALUMBRADO                               |
|ANUNCIOS                                |
|APOYO SERVICIOS FUNERARIOS              |
|ASESORIAS                               |
|ASESORÍA DE TERCEROS ACREDITADOS        |
|ASESORÍA JURÍDICA                       |
|ASISTENCIA SOCIAL                       |
|ASISTENCIA VETERINARIA                  |
|AVISO PREVIO DE MANIFESTACIONES         |
|BACHEO                                  |
|BALIZAMIENTO                            |
|BARBECHO / CHAPONEO                     |
|BECAS                                   |
|CHATARRIZACIÓN                          |
|CONSTRUCCIONES Y OBRAS                  |
|COVID                                   |
|CRÉDITO DE VIVIENDA                     |
|CULTURA Y 

In [23]:
datos_tiempo.groupBy(["tema_solicitud"])\
    .count()\
    .orderBy("tema_solicitud")\
    .show(100, truncate=False)

+----------------------------------------+------+
|tema_solicitud                          |count |
+----------------------------------------+------+
|AGUA POTABLE Y SERVICIOS HIDRÁULICOS    |17573 |
|ALARMAS VECINALES                       |1410  |
|ALERTA SISMICA                          |44182 |
|ALUMBRADO                               |259590|
|ANUNCIOS                                |848   |
|APOYO SERVICIOS FUNERARIOS              |164   |
|ASESORIAS                               |7489  |
|ASESORÍA DE TERCEROS ACREDITADOS        |407   |
|ASESORÍA JURÍDICA                       |3522  |
|ASISTENCIA SOCIAL                       |17254 |
|ASISTENCIA VETERINARIA                  |1747  |
|AVISO PREVIO DE MANIFESTACIONES         |149   |
|BACHEO                                  |96571 |
|BALIZAMIENTO                            |11881 |
|BARBECHO / CHAPONEO                     |22350 |
|BECAS                                   |3195  |
|CHATARRIZACIÓN                          |2843  |


In [24]:
datos_tiempo.groupBy(["tema_solicitud"])\
    .count()\
    .orderBy("count",ascending=False)\
    .show(100, truncate=False)

+----------------------------------------+------+
|tema_solicitud                          |count |
+----------------------------------------+------+
|OTRO                                    |303078|
|ALUMBRADO                               |259590|
|PODA / RETIRO ARBOL                     |214579|
|FALTA DE AGUA                           |205785|
|FUGA DE AGUA                            |109632|
|BACHEO                                  |96571 |
|MANTENIMIENTO VÍA PÚBLICA               |95159 |
|DESAZOLVE                               |93337 |
|RETIRO CASCAJO, ESCOMBRO, AZOLVE, RAMAS |75400 |
|MANTENIMIENTO DE COLADERA / ALCANTARILLA|54717 |
|TRAMITES / INFO VEHICULAR               |46176 |
|ALERTA SISMICA                          |44182 |
|QUEJA DE TRANSPORTE PUBLICO             |42056 |
|VERIFICACIÓN ADMINISTRATIVA             |40880 |
|COVID                                   |37983 |
|RECOLECCIÓN BASURA                      |37071 |
|VEHÍCULO ABANDONADO / CHATARRIZACIÓN    |36450 |


In [25]:
datos_tiempo.groupBy(["anio", "tema_solicitud"])\
    .count()\
    .orderBy(["tema_solicitud", "anio"],ascending=True)\
    .show(100, truncate=False)

+----+------------------------------------+-----+
|anio|tema_solicitud                      |count|
+----+------------------------------------+-----+
|2019|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|1972 |
|2020|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|4487 |
|2021|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|11114|
|2019|ALARMAS VECINALES                   |233  |
|2020|ALARMAS VECINALES                   |205  |
|2021|ALARMAS VECINALES                   |110  |
|2022|ALARMAS VECINALES                   |549  |
|2023|ALARMAS VECINALES                   |301  |
|2024|ALARMAS VECINALES                   |12   |
|2019|ALERTA SISMICA                      |1900 |
|2020|ALERTA SISMICA                      |5379 |
|2021|ALERTA SISMICA                      |25801|
|2022|ALERTA SISMICA                      |7849 |
|2023|ALERTA SISMICA                      |3008 |
|2024|ALERTA SISMICA                      |245  |
|2019|ALUMBRADO                           |37009|
|2020|ALUMBRADO                           |42359|


In [26]:
datos_tiempo.groupBy(["anio", "tema_solicitud"])\
    .count()\
    .orderBy(["tema_solicitud", "anio"],ascending=[True, False])\
    .show(100, truncate=False)

+----+------------------------------------+-----+
|anio|tema_solicitud                      |count|
+----+------------------------------------+-----+
|2021|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|11114|
|2020|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|4487 |
|2019|AGUA POTABLE Y SERVICIOS HIDRÁULICOS|1972 |
|2024|ALARMAS VECINALES                   |12   |
|2023|ALARMAS VECINALES                   |301  |
|2022|ALARMAS VECINALES                   |549  |
|2021|ALARMAS VECINALES                   |110  |
|2020|ALARMAS VECINALES                   |205  |
|2019|ALARMAS VECINALES                   |233  |
|2024|ALERTA SISMICA                      |245  |
|2023|ALERTA SISMICA                      |3008 |
|2022|ALERTA SISMICA                      |7849 |
|2021|ALERTA SISMICA                      |25801|
|2020|ALERTA SISMICA                      |5379 |
|2019|ALERTA SISMICA                      |1900 |
|2024|ALUMBRADO                           |7101 |
|2023|ALUMBRADO                           |55623|


In [27]:
datos_tiempo.filter((datos_tiempo.anio == 2020) & (datos_tiempo.mes <= 6)).count()

167403

In [28]:
datos_tiempo\
    .filter((datos_tiempo.anio == 2020) & (datos_tiempo.mes <= 6))\
    .filter(datos_tiempo.nombre_dia == 'Sunday')\
    .count()

9103

In [29]:
datos_tiempo\
    .select(['edad', 'anio', 'nombre_dia', 'hora', 'estatus'])\
    .filter((datos_tiempo.anio == 2020) & (datos_tiempo.mes <= 6))\
    .filter(datos_tiempo.nombre_dia == 'Sunday')\
    .select(['edad', 'hora', 'estatus'])\
    .filter(datos_tiempo.edad > 0)\
    .show()

+----+----+--------+
|edad|hora| estatus|
+----+----+--------+
|43.0|   0|ATENDIDO|
|50.0|   0|ATENDIDO|
|62.0|   0|ATENDIDO|
|39.0|   0|ATENDIDO|
|65.0|   0|ATENDIDO|
|28.0|   0| TURNADO|
|24.0|   0|ATENDIDO|
|60.0|   0|ATENDIDO|
|47.0|   0|ATENDIDO|
|55.0|   0|ATENDIDO|
|44.0|   0|ATENDIDO|
|40.0|   0|ATENDIDO|
|25.0|   0|ATENDIDO|
|47.0|   0|ATENDIDO|
|45.0|   0|ATENDIDO|
|56.0|   0|ATENDIDO|
|72.0|   0|ATENDIDO|
|44.0|   0|ATENDIDO|
|32.0|   0|ATENDIDO|
| 1.0|   0|ATENDIDO|
+----+----+--------+
only showing top 20 rows



In [30]:
datos.groupBy("alcaldia").count().show(truncate=False)

+----------------------+------+
|alcaldia              |count |
+----------------------+------+
|XOCHIMILCO            |50073 |
|LA MAGDALENA CONTRERAS|8593  |
|AZCAPOTZALCO          |95189 |
|COYOACAN              |86803 |
|TLALPAN               |54937 |
|NA                    |856239|
|TLAHUAC               |17358 |
|CUAJIMALPA DE MORELOS |7496  |
|BENITO JUAREZ         |35919 |
|MILPA ALTA            |2879  |
|GUSTAVO A. MADERO     |84008 |
|IZTAPALAPA            |39777 |
|IZTACALCO             |48242 |
|IZTACALCO             |3573  |
|CUAUHTEMOC            |113121|
|VENUSTIANO CARRANZA   |16713 |
|MIGUEL HIDALGO        |97139 |
|ALVARO OBREGON        |74117 |
|13610                 |322   |
|02090                 |606   |
+----------------------+------+
only showing top 20 rows



In [31]:
datos.groupBy("alcaldia").agg({"edad":"avg"}).show(100, truncate=False)

+----------------------+------------------+
|alcaldia              |avg(edad)         |
+----------------------+------------------+
|XOCHIMILCO            |44.02038006058937 |
|LA MAGDALENA CONTRERAS|34.029847908745246|
|AZCAPOTZALCO          |41.67182940516274 |
|COYOACAN              |39.68447158756239 |
|TLALPAN               |39.53971856559237 |
|NA                    |46.65257871998707 |
|TLAHUAC               |27.59472115793955 |
|CUAJIMALPA DE MORELOS |34.80641025641026 |
|BENITO JUAREZ         |39.11358659430639 |
|MILPA ALTA            |33.32932330827067 |
|GUSTAVO A. MADERO     |38.2184632034632  |
|IZTAPALAPA            |34.50265691797515 |
|IZTACALCO             |41.88917355371901 |
|IZTACALCO             |41.80426861009891 |
|CUAUHTEMOC            |36.632074396182475|
|VENUSTIANO CARRANZA   |34.12961033548656 |
|MIGUEL HIDALGO        |4.992536800110057 |
|ALVARO OBREGON        |38.06759460386418 |
|13610                 |59.971014492753625|
|02090                 |56.43181

In [32]:
dft = datos_tiempo\
    .select(['edad', 'anio', 'nombre_dia', 'hora', 'estatus'])\
    .filter((datos_tiempo.anio == 2020) & (datos_tiempo.mes <= 6))\
    .filter(datos_tiempo.nombre_dia == 'Sunday')\
    .select(['edad', 'hora', 'estatus'])\
    .filter(datos_tiempo.edad > 0)

In [34]:
dft.write.csv("hdfs://namenode:9000/tmp/amd/locateltiempo")

AnalysisException: [PATH_ALREADY_EXISTS] Path hdfs://namenode:9000/tmp/amd/locateltiempo already exists. Set mode as "overwrite" to overwrite the existing path.

In [35]:
dft.printSchema()

root
 |-- edad: float (nullable = true)
 |-- hora: integer (nullable = true)
 |-- estatus: string (nullable = true)



In [37]:
#overwrite, append, ignore

dft.write.mode("ignore").csv("hdfs://namenode:9000/tmp/amd/locateltiempo")

In [38]:
type(dft)

pyspark.sql.dataframe.DataFrame

In [41]:
dft.select("*", (dft.edad * 12).alias("edad_meses")).show(100, truncate = False)

+----+----+--------+----------+
|edad|hora|estatus |edad_meses|
+----+----+--------+----------+
|43.0|0   |ATENDIDO|516.0     |
|50.0|0   |ATENDIDO|600.0     |
|62.0|0   |ATENDIDO|744.0     |
|39.0|0   |ATENDIDO|468.0     |
|65.0|0   |ATENDIDO|780.0     |
|28.0|0   |TURNADO |336.0     |
|24.0|0   |ATENDIDO|288.0     |
|60.0|0   |ATENDIDO|720.0     |
|47.0|0   |ATENDIDO|564.0     |
|55.0|0   |ATENDIDO|660.0     |
|44.0|0   |ATENDIDO|528.0     |
|40.0|0   |ATENDIDO|480.0     |
|25.0|0   |ATENDIDO|300.0     |
|47.0|0   |ATENDIDO|564.0     |
|45.0|0   |ATENDIDO|540.0     |
|56.0|0   |ATENDIDO|672.0     |
|72.0|0   |ATENDIDO|864.0     |
|44.0|0   |ATENDIDO|528.0     |
|32.0|0   |ATENDIDO|384.0     |
|1.0 |0   |ATENDIDO|12.0      |
|25.0|0   |TURNADO |300.0     |
|41.0|0   |ATENDIDO|492.0     |
|47.0|0   |ATENDIDO|564.0     |
|1.0 |0   |ATENDIDO|12.0      |
|1.0 |0   |ATENDIDO|12.0      |
|67.0|0   |ATENDIDO|804.0     |
|25.0|0   |ATENDIDO|300.0     |
|55.0|0   |ATENDIDO|660.0     |
|30.0|0 

In [45]:
dft.select("*", (dft.edad * 12).alias("edad_meses")).write.mode("append").csv("hdfs://namenode:9000/tmp/amd/locateltiempo", header=True)

In [48]:
datos_tiempo.select(["edad", "estatus", "tema_solicitud", "anio", "mes"]).show(truncate=False)

+----+--------+---------------------------------------+----+---+
|edad|estatus |tema_solicitud                         |anio|mes|
+----+--------+---------------------------------------+----+---+
|NULL|ATENDIDO|FUGA DE AGUA                           |2020|1  |
|NULL|ATENDIDO|VENTA DE ALCOHOL / DROGA               |2020|1  |
|NULL|ATENDIDO|TRAMITES / INFO VEHICULAR              |2020|1  |
|NULL|ATENDIDO|MANTENIMIENTO DRENAJE                  |2020|1  |
|NULL|ATENDIDO|SOLICITUD DE VIGILANCIA                |2020|1  |
|NULL|ATENDIDO|ALERTA SISMICA                         |2020|1  |
|NULL|ATENDIDO|VENTA DE ALCOHOL / DROGA               |2020|1  |
|NULL|ATENDIDO|RETIRO CASCAJO, ESCOMBRO, AZOLVE, RAMAS|2020|1  |
|NULL|ATENDIDO|SOLICITUD/SEGURO DE DESEMPLEO          |2020|3  |
|NULL|ATENDIDO|FALTA DE AGUA                          |2020|2  |
|NULL|ATENDIDO|TRAMITES / INFO VEHICULAR              |2020|2  |
|NULL|ATENDIDO|SOLICITUD DE VIGILANCIA                |2020|2  |
|NULL|ATENDIDO|COVID     

In [50]:
from pyspark.sql.functions import count

datos_tiempo.groupBy("anio").agg(count("*").alias("nr")).show(100, truncate=False)

+----+------+
|anio|nr    |
+----+------+
|2020|363065|
|2021|561313|
|2022|500940|
|2023|569362|
|2019|215262|
|2024|113147|
+----+------+



In [51]:
datos_tiempo.groupBy("anio").agg(count("*").alias("nr"))\
.write.mode("append")\
.partitionBy("anio")\
.csv("hdfs://namenode:9000/tmp/amd/locateltiempobyanio",header=True)

In [53]:
datos_tiempo.select(["edad", "estatus", "tema_solicitud", "anio", "mes"])\
.write.mode("append")\
.partitionBy("anio","mes")\
.csv("hdfs://namenode:9000/tmp/amd/locateltiempobyaniomes",header=True)

In [54]:
dfp = spark.read.option("header", True).csv("hdfs://namenode:9000/tmp/amd/locateltiempobyaniomes")

In [55]:
dfp.printSchema()

root
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)



In [56]:
dfp.show()

+----+--------+--------------------+----+---+
|edad| estatus|      tema_solicitud|anio|mes|
+----+--------+--------------------+----+---+
|NULL|ATENDIDO|VEHÍCULOS AUTOMOT...|2021|  9|
|NULL|ATENDIDO| PODA / RETIRO ARBOL|2021|  9|
|NULL|ATENDIDO|        FUGA DE AGUA|2021|  9|
|NULL|ATENDIDO| PODA / RETIRO ARBOL|2021|  9|
|NULL|ATENDIDO|MANTENIMIENTO DE ...|2021|  9|
|NULL|ATENDIDO|      ALERTA SISMICA|2021|  9|
|NULL|ATENDIDO|CONSTRUCCIONES Y ...|2021|  9|
|NULL|ATENDIDO|AGUA POTABLE Y SE...|2021|  9|
|NULL|ATENDIDO|        FUGA DE AGUA|2021|  9|
|NULL|ATENDIDO|       FALTA DE AGUA|2021|  9|
|NULL|ATENDIDO|MANTENIMIENTO DRE...|2021|  9|
|NULL|ATENDIDO|RETIRO CASCAJO, E...|2021|  9|
|NULL|ATENDIDO|MANTENIMIENTO SEM...|2021|  9|
|NULL|ATENDIDO|       FALTA DE AGUA|2021|  9|
|NULL|ATENDIDO|  RECOLECCIÓN BASURA|2021|  9|
|NULL|ATENDIDO|        FUGA DE AGUA|2021|  9|
|29.0|ATENDIDO|               COVID|2021|  9|
|NULL|ATENDIDO|           ALUMBRADO|2021|  9|
|46.0|ATENDIDO|           ASESORIA

In [57]:
dfp.filter("anio = 2022").show()

+----+-------+--------------------+----+---+
|edad|estatus|      tema_solicitud|anio|mes|
+----+-------+--------------------+----+---+
|NULL|CERRADO|RETIRO CASCAJO, E...|2022|  9|
|NULL|TURNADO|       FALTA DE AGUA|2022|  9|
|41.0|CERRADO|QUEJA DE TRANSPOR...|2022|  9|
|NULL|CERRADO|              BACHEO|2022|  9|
|NULL|CERRADO|       PAVIMENTACION|2022|  9|
|48.0|TURNADO|           ALUMBRADO|2022|  9|
|NULL|CERRADO|   QUEJA FUNCIONARIO|2022|  9|
|NULL|CERRADO| PODA / RETIRO ARBOL|2022|  9|
|NULL|CERRADO|           ALUMBRADO|2022|  9|
|NULL|CERRADO|           ALUMBRADO|2022|  9|
|NULL|CERRADO|MANTENIMIENTO VÍA...|2022|  9|
|NULL|TURNADO|              BACHEO|2022|  9|
|NULL|TURNADO|           ALUMBRADO|2022|  9|
|NULL|TURNADO|MANTENIMIENTO VÍA...|2022|  9|
|NULL|CERRADO|           ALUMBRADO|2022|  9|
|NULL|TURNADO|SOLICITUD DE VIGI...|2022|  9|
|NULL|CERRADO|QUEJA DE TRANSPOR...|2022|  9|
|NULL|CERRADO|           ALUMBRADO|2022|  9|
|50.0|CERRADO|MANTENIMIENTO SEM...|2022|  9|
|NULL|CERR

In [58]:
dfp.filter("anio = 2022 AND mes = 7").show()

+----+-------+--------------------+----+---+
|edad|estatus|      tema_solicitud|anio|mes|
+----+-------+--------------------+----+---+
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|SOLICITUD EVALUAC...|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|SOLICITUD/SEGURO ...|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|MANTENIMIENTO PAR...|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|INFORMACIÓN SOBRE...|2022|  7|
|NULL|CERR

In [59]:
dfp.filter("mes = 7 AND anio = 2022").show()

+----+-------+--------------------+----+---+
|edad|estatus|      tema_solicitud|anio|mes|
+----+-------+--------------------+----+---+
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|SOLICITUD EVALUAC...|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|SOLICITUD/SEGURO ...|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|MANTENIMIENTO PAR...|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|      REGISTRO CIVIL|2022|  7|
|NULL|CERRADO|TRAMITES / INFO V...|2022|  7|
|NULL|CERRADO|INFORMACIÓN SOBRE...|2022|  7|
|NULL|CERR